In [5]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.metrics import precision_score

In [6]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            self.df_ratings = pd.read_csv(rating_data, low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))
            
    
    def random_group(self, n):
        self.users_list = list(self.df_ratings['userId'])
        random_group = random.sample(self.users_list,n)
        return random_group
        

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('../datasets/ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.8):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie['movieID']].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
#                 print("sim_score_title= {}\t sim_score_genres= {}".format(sim_scores_title[i][1], sim_scores_genres[i][1]))
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
#                 print("sim_score_total= {}".format(total_sim_score))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            self.total_sim_score = total_sim_score
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs
    
    
    def get_relevance_score(self, recs, references):
        count = 0
        recs_dict = []
        for reference in references:
        #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

            for movie in recs[count]:
                aux = {}

                movie_id = self.movies.loc[movie[0]].values[0]
                movie_title = self.movies.loc[movie[0]].values[1]
                movie_genres = self.movies.loc[movie[0]].values[2]
                movie_similarity = movie[1]
                movie_relevance = round(((reference['rating']/5.0)+movie_similarity)/2, 3)

                aux['movie_id'] = movie_id
                aux['movie_title'] = movie_title
                aux['movie_genres'] = movie_genres
                aux['movie_similarity'] = movie_similarity
                aux['movie_relevance'] = movie_relevance

                recs_dict.append(aux)

        #         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

            count=count+1

        recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

        return recs_dict
    
    
    def calc_distance_item_in_list(self, item, this_list, title_weight=0.8):

        idx_i = int(self.movies[self.movies['movieId']==item['movie_id']].index[0])

        total_dist = 0
        for movie in this_list:
            
            idx_j = int(self.movies[self.movies['movieId']==int(movie['movie_id'])].index[0])

            sim_i_j = (self.cosine_sim_movies_title[idx_i][idx_j]*title_weight) + (self.cosine_sim_movies_genres[idx_i][idx_j]*(1-title_weight))
            dist_i_j = 1 - sim_i_j
            total_dist = total_dist + dist_i_j

        result = total_dist/len(this_list)
        return result
    
    
    def calc_diversity_score(self, actual_list, candidates_list, alfa=0.5):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        func(i,R) = (relevance[i]*alfa) + (dist_i_R(i,R)*(1-alfa))
        '''
        diversity_score = []
        count = 0

        for item in candidates_list:

            aux = {}
            dist_item_R = self.calc_distance_item_in_list(item=item, this_list=actual_list)
            aux['div_score'] = (item['movie_relevance']*alfa) + (dist_item_R*(1-alfa))
            aux['idx'] = count
            diversity_score.append(aux)
            count = count + 1

        return diversity_score
    
    
    def diversify_recs_list(self, recs, n=10):
        '''
        This function implemented here was based on MARIUS KAMINSKAS and DEREK BRIDGE paper: Diversity, Serendipity, Novelty, and Coverage: A Survey and Empirical Analysis of Beyond-Accuracy Objectives in Recommender Systems
        The Greedy Reranking Algorithm.
        '''
        diversified_list = []
        
        while len(diversified_list) < n:
            if len(diversified_list) == 0:
                diversified_list.append(recs[0])
                recs.pop(0)
            else:
                diversity_score = self.calc_diversity_score(actual_list=diversified_list, candidates_list=recs)
                diversity_score = sorted(diversity_score, key = lambda i: i['div_score'],reverse=True)
#               #  Add the item that maximize diversity in the list 
                item = diversity_score[0]
                diversified_list.append(recs[item['idx']])
#               #  Remove this item from the candidates list
                recs.pop(item['idx'])
    
        return diversified_list
    
    
    def divesify_recs_list_bounded_random(self, recs, n=10):
        '''
        This function implemented here was based on KEITH BRADLEY and BARRY SMYTH paper: Improving Recommendation Diversity
        The Bounded Random Selection Algorithm.
        '''
        diversified_list = random.sample(recs,n)

        return diversified_list
                
        
#     def get_ILS(self, final_recs, title_weight=0.8):
#         big_list = []
#         for i in final_recs:
#             movie_idx_i = int(self.movies[self.movies['movieId']==i['movie_id']].index[0])
#             small_list = []
#             for j in final_recs:
#                 movie_idx_j = int(self.movies[self.movies['movieId']==j['movie_id']].index[0])
#                 sim_genre = self.cosine_sim_movies_genres[movie_idx_i][movie_idx_j]
#                 sim_title = self.cosine_sim_movies_title[movie_idx_i][movie_idx_j]
#                 total_sim = (sim_title*title_weight) + (sim_genre*(1-title_weight))
#                 small_list.append(total_sim)
#             big_list.append(small_list)

#         return big_list

    def calc_dist_i_j(self, idx_i, idx_j, title_weight=0.8):
        sim_genre = self.cosine_sim_movies_genres[idx_i][idx_j]
        sim_title = self.cosine_sim_movies_title[idx_i][idx_j]
        total_sim = (sim_title*title_weight) + (sim_genre*(1-title_weight))
        dist_score = 1 - total_sim

        return dist_score
    
    
    def get_distance_matrix(self, final_recs, title_weight=0.8):
        dist_matrix = []
        for i in final_recs:
            aux = []
            movie_idx_i = int(self.movies[self.movies['movieId']==i['movie_id']].index[0])
            for j in final_recs:
                movie_idx_j = int(self.movies[self.movies['movieId']==j['movie_id']].index[0])
                dist_i_j = self.calc_dist_i_j(movie_idx_i, movie_idx_j, title_weight=0.8)
                aux.append(dist_i_j)
            dist_matrix.append(aux)
            
        return dist_matrix
    
    def get_ILD_score(self, final_recs, title_weight=0.8):
        dist_matrix = self.get_distance_matrix(final_recs, title_weight=0.8)
        np_dist_mtx = np.array(dist_matrix)
        upper_right = np.triu_indices(np_dist_mtx.shape[0], k=1)

        ild_score = np.mean(np_dist_mtx[upper_right])
        
        return ild_score
    
    
    
    # # # # # # # # # # PRECISION Module # # # # # # # # # #
    
    def get_mean(self, movie):
        converted_values = []
        for item in movie['ratings']:
            for bla in item:
                aux = float(bla)
                converted_values.append(aux)

        my_mean = sum(converted_values) / len(converted_values)
        my_mean = round(my_mean, 3)
        return my_mean
    
    
    def get_movies_means(self, movies_list, at):
        my_copy = self.df_ratings.copy()

        df_movies_ratings = my_copy.groupby('movieId')['rating'].apply(list).reset_index(name='ratings')

        movies_means = []

        for item in movies_list[:at]:
            movie = df_movies_ratings[df_movies_ratings['movieId']==item['movie_id']]
            movies_means.append(self.get_mean(movie))

        return movies_means
    
    
    def binary_mean(self, movies_mean, cutoff):
        binary_mean = []
        returned_movies = []
        for item in movies_mean:
            if item >= cutoff:
                binary_mean.append(1)
            else:
                binary_mean.append(0)

            returned_movies.append(1)

        return precision_score(binary_mean, returned_movies)
    
    
    def precision_at_offline(self, movies_list, at):
    
        global_mean = self.trainset.global_mean
        movies_list_mean = self.get_movies_means(movies_list, at)

        print("Global mean: {}, movies_list_mean: {}".format(global_mean, movies_list_mean))

        precision = self.binary_mean(movies_list_mean, global_mean)
        return precision

In [7]:
refinedMyAlgo = RefinedMyAlgo(rating_data='../datasets/ml-latest-small/ratings.csv', movie_data='../datasets/ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [8]:
refinedMyAlgo.calc_similarity_matrix()

In [9]:
# # # FIXED GROUP
# my_group = [77,596,452,243,420]

# # # RANDOM GROUP
my_group = refinedMyAlgo.random_group(5)
print(my_group)

refinedMyAlgo.predict_ratings(users=my_group)
print(len(refinedMyAlgo.predictions))

[488, 226, 602, 52, 68]
46567


In [10]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_group)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_group)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,2,3,5,6,7,10,11,14,16,...,134853,135143,139385,142488,152081,164179,166528,166635,168252,176371
userId,,,,,,,,,,,,,,,,,,,,,
52,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,...,5,0,0,0.0,5,0.0,0.0,0,0,0.0
68,2.5,2.5,2.0,2,4,2,4.5,4.5,0,3.5,...,0,4,3,4.5,3,3.5,4.5,3,4,3.5
226,3.5,3.0,3.5,0,0,0,4.0,0.0,0,4.5,...,0,0,0,0.0,0,0.0,0.0,0,0,0.0
488,4.5,0.0,0.0,0,0,0,0.0,0.0,0,0.0,...,0,0,0,0.0,0,0.0,0.0,0,0,0.0
602,0.0,4.0,0.0,0,3,0,3.0,3.0,5,3.0,...,0,0,0,0.0,0,0.0,0.0,0,0,0.0


In [11]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx = group_filled_mtx.round(decimals=3)
group_filled_mtx.head()

movieId,1,2,3,5,6,7,10,11,14,16,...,134853,135143,139385,142488,152081,164179,166528,166635,168252,176371
userId,,,,,,,,,,,,,,,,,,,,,
52,4.763,4.041,3.731,3.719,4.643,3.859,4.035,4.346,4.337,4.715,...,5.000,4.119,4.539,4.689,5.000,4.446,4.514,4.127,4.291,4.530
68,2.500,2.500,2.000,2.000,4.000,2.000,4.500,4.500,3.747,3.500,...,3.537,4.000,3.000,4.500,3.000,3.500,4.500,3.000,4.000,3.500
226,3.500,3.000,3.500,2.579,4.030,2.789,4.000,3.414,3.539,4.500,...,3.779,3.488,3.900,4.060,3.733,3.729,3.748,3.318,4.134,3.757
488,4.500,3.241,3.283,3.023,4.168,3.239,3.416,3.731,3.743,3.907,...,3.764,3.616,3.819,4.208,3.599,4.017,3.806,3.434,4.085,3.809
602,3.805,4.000,2.865,2.906,3.000,2.908,3.000,3.000,5.000,3.000,...,3.594,3.544,3.703,4.051,3.450,3.692,3.733,3.438,3.888,3.426


In [7]:
########################################################################
# # Implementing LEAST MISERY ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,3.745,3.344,3.5,3.652,3.647,3.422,2.706,3.997,3.123,3.5,...,2.5,3.251,3.5,3.701,3.173,3.5,3.339,3.5,3.358,3.579


In [7]:
########################################################################
# # Implementing MOST PLEASURE ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(max(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

,1,2,6,10,11,13,16,32,34,36,...,175401,175431,175435,176101,177593,178613,179133,180095,180777,185135
900,4.5,4.359,4.3,4.162,4.358,3.937,4.5,4.174,4.576,4.273,...,4,5,4.5,3.849,5,4.072,5,4.404,4.5,4.5


In [12]:
########################################################################
# # Implementing AVERAGE WITHOUT MISERY: treshold=2  ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    if float(min(my_col)) <= 2 :
        values.append( float(min(my_col)) )
    else:
        values.append( float( sum(my_col) / len(my_col) ) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

    
agg_group_perf = agg_group_perf.round(decimals=3)
agg_group_perf.head()

,1,2,3,5,6,7,10,11,14,16,...,134853,135143,139385,142488,152081,164179,166528,166635,168252,176371
900,3.8136,3.3564,2,2,3.9682,2,3.7902,3.7982,4.0732,3.9244,...,3.9348,3.7534,3.7922,4.3016,3.7564,3.8768,4.0602,3.4634,4.0796,3.8044


In [13]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.7316, 'movieID': 1197},
 {'rating': 4.614, 'movieID': 7153},
 {'rating': 4.5771999999999995, 'movieID': 6711},
 {'rating': 4.5638000000000005, 'movieID': 58559},
 {'rating': 4.5552, 'movieID': 47},
 {'rating': 4.549, 'movieID': 1210},
 {'rating': 4.5452, 'movieID': 4306},
 {'rating': 4.5256, 'movieID': 527},
 {'rating': 4.5226, 'movieID': 8368},
 {'rating': 4.5043999999999995, 'movieID': 904},
 {'rating': 4.5026, 'movieID': 2502},
 {'rating': 4.4996, 'movieID': 260},
 {'rating': 4.488, 'movieID': 318},
 {'rating': 4.4536, 'movieID': 1136},
 {'rating': 4.422199999999999, 'movieID': 79132},
 {'rating': 4.418200000000001, 'movieID': 4262},
 {'rating': 4.416600000000001, 'movieID': 1291},
 {'rating': 4.416, 'movieID': 912},
 {'rating': 4.4078, 'movieID': 49272},
 {'rating': 4.4068000000000005, 'movieID': 55118},
 {'rating': 4.4054, 'movieID': 593},
 {'rating': 4.3946, 'movieID': 1196},
 {'rating': 4.393199999999999, 'movieID': 6377},
 {'rating': 4.3904, 'movieID': 720},
 {'ra

In [14]:
references = group_pref_dict[0:10]
# references = group_pref_dict

for item in references:
    print(item)
# print(references)

{'rating': 4.7316, 'movieID': 1197}
{'rating': 4.614, 'movieID': 7153}
{'rating': 4.5771999999999995, 'movieID': 6711}
{'rating': 4.5638000000000005, 'movieID': 58559}
{'rating': 4.5552, 'movieID': 47}
{'rating': 4.549, 'movieID': 1210}
{'rating': 4.5452, 'movieID': 4306}
{'rating': 4.5256, 'movieID': 527}
{'rating': 4.5226, 'movieID': 8368}
{'rating': 4.5043999999999995, 'movieID': 904}


In [15]:
# references=[904, 318, 38061, 31658, 57669]
recs = refinedMyAlgo.get_similar_movies(references)

In [16]:
# print(refinedMyAlgo.total_sim_score)

In [17]:
candidates_list = refinedMyAlgo.get_relevance_score(recs=recs, references=references)
print(len(candidates_list))

for item in candidates_list[0:20]:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

100
relevance: 0.846, title:Shrek Forever After (a.k.a. Shrek: The Final Chapter)
relevance: 0.8, title:Shrek the Halls
relevance: 0.75, title:Batman: The Dark Knight Returns, Part 1
relevance: 0.737, title:Cry in the Dark, A
relevance: 0.729, title:Bride Wars
relevance: 0.726, title:Alone in the Dark
relevance: 0.725, title:Knight and Day
relevance: 0.706, title:Star Wars: The Clone Wars
relevance: 0.704, title:Father of the Bride
relevance: 0.704, title:All Is Lost
relevance: 0.699, title:Decoy Bride, The
relevance: 0.695, title:Black Knight
relevance: 0.693, title:Dark Blue
relevance: 0.692, title:Little Princess, A
relevance: 0.692, title:Little Princess, The
relevance: 0.691, title:Bride & Prejudice
relevance: 0.691, title:Princess and the Frog, The
relevance: 0.686, title:I Was a Male War Bride
relevance: 0.686, title:Ice Princess
relevance: 0.683, title:In the Name of the King III


In [20]:
my_candidates = candidates_list.copy()
final_recs_greedy = refinedMyAlgo.diversify_recs_list(recs=my_candidates)
for item in final_recs_greedy:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

relevance: 0.846, title:Shrek Forever After (a.k.a. Shrek: The Final Chapter)
relevance: 0.737, title:Cry in the Dark, A
relevance: 0.729, title:Bride Wars
relevance: 0.725, title:Knight and Day
relevance: 0.657, title:Kill List
relevance: 0.8, title:Shrek the Halls
relevance: 0.692, title:Little Princess, A
relevance: 0.704, title:All Is Lost
relevance: 0.663, title:Star Trek VI: The Undiscovered Country
relevance: 0.635, title:We Could Be King


In [24]:
my_candidates = candidates_list.copy()
final_recs_random = refinedMyAlgo.divesify_recs_list_bounded_random(recs=my_candidates)
for item in final_recs_random:
    print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

relevance: 0.665, title:Lost in America
relevance: 0.695, title:Black Knight
relevance: 0.55, title:Pacific Heights
relevance: 0.553, title:Paths of Glory
relevance: 0.646, title:Seven Sisters
relevance: 0.553, title:Land and Freedom (Tierra y libertad)
relevance: 0.55, title:True Crime
relevance: 0.64, title:Seven-Per-Cent Solution, The
relevance: 0.637, title:Return to Oz
relevance: 0.574, title:Very Potter Sequel, A


In [2]:
# dist_i_R = refinedMyAlgo.calc_distance_item_in_list(item=candidates_list[1],this_list=candidates_list[0:5])
# dist_i_R

In [3]:
# print(refinedMyAlgo.calc_diversity_score(actual_list=candidates_list[0:5], candidates_list=candidates_list[10:20]))

In [4]:
# my_candidates = candidates_list.copy()
# final_recs = refinedMyAlgo.diversify_recs_list(recs=my_candidates)
# for item in final_recs:
#     print('relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

In [22]:
# standard_recs = candidates_list[0:10]
# refinedMyAlgo.get_ILD_score(standard_recs, title_weight=0.8)

In [25]:
standard_recs = candidates_list[0:10]
print('ILD - standard recs: {}'.format(refinedMyAlgo.get_ILD_score(standard_recs, title_weight=0.8)))
print('ILD - div greedy algo: {}'.format(refinedMyAlgo.get_ILD_score(final_recs_greedy, title_weight=0.8)))
print('ILD - div random algo: {}'.format(refinedMyAlgo.get_ILD_score(final_recs_random, title_weight=0.8)))

ILD - standard recs: 0.885588789271037
ILD - div greedy algo: 0.9673954785321026
ILD - div random algo: 0.9632280935816209


In [26]:
print('P@10 - standard recs: {}'.format(refinedMyAlgo.precision_at_offline(standard_recs, 10)))
print('P@10 - div greedy algo: {}'.format(refinedMyAlgo.precision_at_offline(final_recs_greedy, 10)))
print('P@10 - div random algo: {}'.format(refinedMyAlgo.precision_at_offline(final_recs_random, 10)))

Global mean: 3.501556983616962, movies_list_mean: [3.571, 2.5, 3.929, 3.75, 3.5, 0.5, 3.417, 2.357, 3.25, 3.625]
P@10 - standard recs: 0.4
Global mean: 3.501556983616962, movies_list_mean: [3.571, 3.75, 3.5, 3.417, 4.5, 2.5, 3.905, 3.625, 3.345, 4.0]
P@10 - div greedy algo: 0.6
Global mean: 3.501556983616962, movies_list_mean: [2.5, 2.188, 3.192, 4.542, 4.25, 4.0, 4.0, 2.5, 3.0, 5.0]
P@10 - div random algo: 0.5


In [19]:
print(refinedMyAlgo.precision_at_offline(final_recs, 10))

Global mean: 3.501556983616962, movies_list_mean: [2.5, 1.0, 4.0, 3.25, 3.75, 3.0, 3.5, 3.875, 4.5, 2.0]
0.4


In [6]:
'''Runing evaluation metrics on BASELINE outputs'''

def parse_to_evaluate(movie_id_list):
    output = []
    for i in movie_id_list:
        cube = {}
        cube['movie_id'] = i
        output.append(cube)
        
    return output

output_1 = parse_to_evaluate([4437,2118,5244,50740,1251,27773,1261,6385,57528,3400])
output_2 = parse_to_evaluate([66934,1223,2498,8605,50685,70183,1381,6155,111743,8493])
output_3 = parse_to_evaluate([5013,68237,7361,902,2109,3623,4446,177,1036,1202])
output_4 = parse_to_evaluate([3361,156781,66798,784,4002,108949,2134,2968,157108,81087])
output_5 = parse_to_evaluate([65261,94018,7132,2648,6870,86290,3623,59725,808,3635])

# print(output_1)

In [22]:
# for item in output_1:
# #     movie = df_movies_ratings[df_movies_ratings['movieId']==item['movie_id']]
#     print(item['movie_id'])

In [7]:
refinedMyAlgo.get_ILD_score(output_1, title_weight=0.8)

0.9694159990485626

In [8]:
refinedMyAlgo.get_ILD_score(output_2, title_weight=0.8)

0.9442230039447174

In [9]:
refinedMyAlgo.get_ILD_score(output_3, title_weight=0.8)

0.9720653556231904

In [10]:
refinedMyAlgo.get_ILD_score(output_4, title_weight=0.8)

0.9563492202572766

In [11]:
refinedMyAlgo.get_ILD_score(output_5, title_weight=0.8)

0.97624502398981

In [23]:
refinedMyAlgo.precision_at_offline(output_1,10)

Global mean: 3.501556983616962, movies_list_mean: [3.938, 3.733, 5.0, 3.833, 4.071, 4.09, 4.044, 3.75, 3.278, 3.0]


0.8

In [24]:
refinedMyAlgo.precision_at_offline(output_2,10)

Global mean: 3.501556983616962, movies_list_mean: [3.917, 4.089, 2.1, 3.25, 3.833, 3.667, 2.079, 3.276, 2.958, 3.083]


0.4

In [25]:
refinedMyAlgo.precision_at_offline(output_3,10)

Global mean: 3.501556983616962, movies_list_mean: [3.704, 3.969, 4.16, 3.847, 3.37, 2.714, 3.355, 2.75, 3.862, 3.857]


0.6

In [26]:
refinedMyAlgo.precision_at_offline(output_4,10)

Global mean: 3.501556983616962, movies_list_mean: [3.671, 4.0, 2.75, 2.806, 3.852, 3.333, 3.114, 3.732, 4.5, 4.5]


0.6

In [27]:
refinedMyAlgo.precision_at_offline(output_5,10)

Global mean: 3.501556983616962, movies_list_mean: [4.0, 2.333, 4.111, 3.909, 3.712, 4.0, 2.714, 2.409, 3.5, 3.444]


0.5